In [1]:
import pandas as pd
import numpy as np
import scipy as sc

In [ ]:
if __name__ == '__main__':
    # Initialize the ARHMM class
    test = ARHMM('phase_mappings_for_hormones_with_adjusted_slope.csv')

   # data loading
    dflh = pd.read_csv('lhfw_interp.csv')
    dfpdg = pd.read_csv('pdgfw_interp.csv')
    dfe3g = pd.read_csv('e3gfw_interp.csv')

    # List of hub_ids and cycle indexes to iterate over
    cycles = [(hub_id, cycle_index) for hub_id, cycle_index in zip(dflh['hub_id'], dflh['cycle_index'])]

    # init empty list
    results = []

    # Iterations through each cycle
    for hub_id, cycle_index in cycles:
        # Get the hormone values for the current cycle
        lh_vals = dflh.loc[(dflh['hub_id'] == hub_id) & (dflh['cycle_index'] == cycle_index)]
        pdg_vals = dfpdg.loc[(dfpdg['hub_id'] == hub_id) & (dfpdg['cycle_index'] == cycle_index)]
        e3g_vals = dfe3g.loc[(dfe3g['hub_id'] == hub_id) & (dfe3g['cycle_index'] == cycle_index)]

        # Extract the cycle length and hormone values
        if not lh_vals.empty and not pdg_vals.empty and not e3g_vals.empty:
            cycle_length = lh_vals['cycle_length'].iloc[0]
            lhvals = lh_vals[['day' + str(i) + '_LH' for i in range(cycle_length)]].iloc[0].tolist()
            pdgvals = pdg_vals[['day' + str(i) + '_PDG' for i in range(cycle_length)]].iloc[0].tolist()
            e3gvals = e3g_vals[['day' + str(i) + '_E3G' for i in range(cycle_length)]].iloc[0].tolist()

            # .forward
            F = test.forward(lhvals, pdgvals, e3gvals, emission_type='all')

            # results and saving file when done
            results.append({
                'hub_id': hub_id,
                'cycle_index': cycle_index,
                'logprob': F['logprob']
            })


    results_df = pd.DataFrame(results)
    results_df.to_csv('cycles_logprob.csv', index=False)
